In [ ]:
import os
import zipfile
import xgboost
import numpy as np
import pandas as pd
import nltk
import spacy
from spacy import displacy
from spacy.matcher import Matcher

from collections import Counter

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import string

import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import numpy as np
import re
import string
from nltk.stem import PorterStemmer
# from gensim.models import word2vec
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
# from tsne import bh_sne
import gensim
from sklearn.metrics import roc_curve, auc, accuracy_score, recall_score, precision_score

from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
stopwords_pl = open("/content/polish.stopwords.txt", "r")
stopwords_pl = stopwords_pl.read()
stopwords_pl=stopwords_pl.split("\n")

In [ ]:
local_zip = "/content/polska_literatura.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
#dataset path

dataset_path = os.path.join('/tmp/polska_literatura')

In [ ]:
ls /tmp/polska_literatura

krasinski/  lesmian/  mickiewicz/  norwid/  prus/  sienkiewicz/  slowacki/


In [ ]:
os.listdir('/tmp/polska_literatura/')

['norwid',
 'slowacki',
 'krasinski',
 'prus',
 'lesmian',
 'sienkiewicz',
 'mickiewicz']

In [ ]:
rm -r /tmp/polska_literatura/.DS_Store

In [ ]:
def create_dataset_author(dir_path, author):
  text_list = []
  for book in os.listdir(dir_path):
    try:
      path = dir_path+"/"+book
      text = open(path, 'r').read()
      text_list.append(text)
    except:
      pass
  df = pd.DataFrame()
  N = len(text_list)
  df['tytul'] = os.listdir(dir_path)[:N]
  df['tekst'] = text_list
  df['autor']=author
  return df


In [ ]:
list_df = []
for author in os.listdir('/tmp/polska_literatura/')[:-1]:
  path = '/tmp/polska_literatura/{author}'.format(author=author)
  temp = create_dataset_author(path, author)
  list_df.append(temp)
  





In [ ]:
df = pd.concat(list_df)

In [ ]:
df = df.reset_index()
df['tytul'] = df['tytul'].apply(lambda x: x.strip(".txt"))

In [ ]:
down = df[df.columns[1:]]

In [ ]:
down

,index,tytul,tekst,autor
0,0,norwid-pielgrzym,1\n\nNad stanami jest i *stanów-stan*.\nJako w...,norwid
1,1,cos-ty-atenom-zrobil-sokratesie,"I\n\nCoś ty Atenom zrobił, Sokratesie,\nŻe ci ...",norwid
2,2,norwid-do-spolczesnych,"I\n\nI pożegnałem kraj, i brzegi znane\nOdepch...",norwid
3,3,piesn-od-ziemi-naszej,"„Et aux horions, l'on verra qui\na meilleur dr...",norwid
4,4,promethidion,"(Plin: O Gladiatorze konającym)\n\n\n\nTobie, ...",norwid
...,...,...,...,...
69,13,bez-dogmatu,"Tom I\n\nRzym, 9 Stycznia\n\nPrzed kilku miesi...",sienkiewicz
70,14,potop-tom-drugi,Tom drugi\n\n\n\nRozdział I\n\nWierny Soroka w...,sienkiewicz
71,15,latarnik,Opowiadanie to osnute jest na wypadku rzeczywi...,sienkiewicz
72,16,sienkiewicz-pojdzmy-za-nim,I\n\nKajus Septimus Cinna był patrycjuszem rzy...,sienkiewicz


In [ ]:
down.to_csv("polish_authors.csv")

In [ ]:
class clean_text:

  def split_text(self, t):
      return t.apply(lambda x: str(x).split(" "))

  def to_lower(self, t):
      return t.apply(lambda x:   str(x).lower())


  def remove_numbers(self, t):
      return t.apply(lambda x: re.sub(r'\d+', '', str(x)))



  def remove_punctuation(self,t):
        return t.apply(lambda x: str(x).translate(str.maketrans('','',string.punctuation)))



  def remove_stopwords_pl(self, t):
      return t.apply(lambda x: [word for word in str(x).split(" ") if word not in stopwords_pl])

  def stemming(self,t):
      temp= t.apply(lambda x: str(x).split(" "))
      porter = PorterStemmer()
      stemmed = temp.apply(lambda x: porter.stem(" ".join(x)))
      return stemmed

  def remove_new_line(self, t):
      return t.apply(lambda x:   str(x).replace("\n", " "))

  # def remove_character(self, t):
  #     return t.apply(lambda x:   str(x).replace("—", " "))

     

  def ready_data(self, t):
      t1 = self.split_text(t)
      t2 = self.to_lower(t1)
      t3 = self.remove_numbers(t2)
      t4 = self.remove_punctuation(t3)
      t5 = self.remove_stopwords_pl(t4)
      t6 = self.stemming(t5)
      t7 = self.remove_new_line(t6)
      # t8 = self.remove_new_line(t7)
      
      return t7

In [ ]:
X= clean_text().ready_data(df.tekst)


In [ ]:
X = list(map(eval, X))

In [ ]:
for i in range(len(X)):
  X[i]=[x.strip('—n') for x in X[i]]
  X[i] = [x for x in X[i] if len(x)>0]

In [ ]:
# X = sum(X, [])

In [ ]:
X = [" ".join(x) for x in X]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(X)

In [ ]:
#create X(features), y(labels)
X = x
y = df.autor

In [ ]:
#label encoding

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y_enc = le.fit_transform(y)


In [ ]:
d= dict(zip(le.transform(le.classes_),le.classes_))
d

{0: 'krasinski',
 1: 'lesmian',
 2: 'mickiewicz',
 3: 'norwid',
 4: 'prus',
 5: 'sienkiewicz',
 6: 'slowacki'}

In [ ]:
def split_data(X,y, ratio):
    from sklearn.model_selection import train_test_split
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=ratio, random_state=42)
    return train_X, test_X, train_y, test_y

In [ ]:
train_X, test_X, train_y, test_y = split_data(X,y_enc ,0.5)

<45x183603 sparse matrix of type '<class 'numpy.float64'>'
	with 231822 stored elements in Compressed Sparse Row format>

In [ ]:
clf = xgboost.XGBClassifier()
clf = clf.fit(train_X, train_y)

predictions = clf.predict(test_X)
prob = clf.predict_proba(test_X) #probalities

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         1
           1     0.5455    0.8571    0.6667         7
           2     0.5000    0.4000    0.4444        10
           3     0.0000    0.0000    0.0000         4
           4     0.8333    1.0000    0.9091         5
           5     0.8000    0.8000    0.8000        10
           6     1.0000    0.6667    0.8000         9

    accuracy                         0.6304        46
   macro avg     0.5255    0.5320    0.5172        46
weighted avg     0.6518    0.6304    0.6273        46



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
report = classification_report(test_y, predictions, digits=4,output_dict=True)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


dict_values(['krasinski', 'lesmian', 'mickiewicz', 'norwid', 'prus', 'sienkiewicz', 'slowacki'])

In [ ]:
report=pd.DataFrame(report)
new_columns = [0, 1, 2, 3, 4, 5, 6, 'accuracy', 'macro avg','weighted avg']
report.columns=new_columns

In [ ]:
report.rename(columns=d, inplace=True)
report

,krasinski,lesmian,mickiewicz,norwid,prus,sienkiewicz,slowacki,accuracy,macro avg,weighted avg
precision,0.0,0.545455,0.500000,0.0,0.833333,0.8,1.000000,0.630435,0.525541,0.651845
recall,0.0,0.857143,0.400000,0.0,1.000000,0.8,0.666667,0.630435,0.531973,0.630435
f1-score,0.0,0.666667,0.444444,0.0,0.909091,0.8,0.800000,0.630435,0.517172,0.627317
support,1.0,7.000000,10.000000,4.0,5.000000,10.0,9.000000,0.630435,46.000000,46.000000


In [ ]:
def metrics(test_y,pred):
		print("Accuracy", accuracy_score(test_y, pred))
		print("recall", recall_score(test_y, pred, average='micro'))
		print("precision", precision_score(test_y, pred, average='micro'))

In [ ]:
metrics(test_y,predictions)

Accuracy 0.6304347826086957
recall 0.6304347826086957
precision 0.6304347826086957
